In [1]:
import pygame

class keys:
    def __init__(self, event= None):
        self.last_pressed = ""
        self.string = "Pressed:"
        if event.key == pygame.K_UP:
            self.UP = True
            self.string += "UP"
        else:
            self.UP = False
        if event.key == pygame.K_DOWN:
            self.DOWN = True
            self.string += "DOWN"
        else:
            self.DOWN = False
        if event.key == pygame.K_RIGHT:
            self.RIGHT = True
            self.string += "RIGHT"
            self.last_pressed = "RIGHT"
        else:
            self.RIGHT = False    
        if event.key == pygame.K_LEFT:
            self.LEFT = True
            self.string += "LEFT"
            self.last_pressed = "LEFT"
        else:
            self.LEFT = False 
 
    def __str__(self):
        return self.string
        
        
def xy_position_to_pixels(Rectangle, window_size, virtual_window):
    """Takes:In-game coordinate rectangle
    Converts in game coordinates to pixels. Also, note that the in-game position refers to the LOWER RIGHT hand
    corner of the sprite. 
    Returns: rectangle in pixel coordinates"""
    x = Rectangle[0]
    y = Rectangle[1]
    width = Rectangle[2]
    height = Rectangle[3]
    pixel_x = int(float(x/virtual_window[0])*window_size[0])
    pixel_y = window_size[1] - int(float(y/virtual_window[1])*window_size[1])
    pixel_width = int(float(width/virtual_window[0])*window_size[0])
    pixel_height = int(float(height/virtual_window[1])*window_size[1])
    pixel_y -= pixel_height
    return (pixel_x, pixel_y, pixel_width, pixel_height)



class position:
    def __init__(self, current_x = 0, current_y = 0, current_xs = 0, current_ys = 0, floor = 0, button=None,
                character_width = 20, character_height = 30, wall_speed = .3):
        self.current_x = current_x
        self.current_y = current_y
        self.current_xs = current_xs
        self.current_ys = current_ys
        self.button = button
        self.floor = floor
        self.last_pressed = None
        self.jump = False
        self.character_width = character_width
        self.character_height = character_height
        self.wall = None
        self.wall_speed = wall_speed
        
    def __str__(self):
        return str(self.current_x)+" "+str(self.current_y)+" "+str(self.current_xs)+" "+str(self.current_ys)
    
    def give_next_position(self, floor, wall): #fix button 
        #print("The give next position function has run.")
        #print("Button:", button)
        self.floor = floor
        self.wall = wall
        
        
        if self.current_y <= self.floor:
            self.current_ys = 0
            self.jump = False
        if not self.jump:    
            self.current_xs = -1*self.wall_speed
        if self.current_y > self.floor:
            self.current_ys -= .5
            
        if self.button != None: 
            #print("AH I SENSE MOVEMENT!")
            if self.current_y == self.floor:
                if self.button.UP:
                    self.current_ys = 10 
                    if self.last_pressed == "RIGHT":
                        self.current_xs = 1
                        self.jump = True
                        if wall[1] != None:
                            if self.current_x+self.character_width+1 >= wall[1]:
                                self.current_xs = 0
                    elif self.last_pressed == "LEFT":
                        self.current_xs = -1
                        self.jump = True
                        if wall[0] != None:
                            if self.current_x <= wall[0]:
                                self.current_xs = 0
                        
            if self.button.RIGHT:
                self.current_xs = 1
                if wall[1] != None:
                    if self.current_x+self.character_width+1 >= wall[1]:
                        self.current_xs = 0
            elif self.button.LEFT:
                self.current_xs = -1
                if wall[0] != None:
                    if self.current_x <= wall[0]:
                        self.current_xs = 0
            self.last_pressed = self.button.last_pressed
        
        self.current_x += self.current_xs
        self.current_y += self.current_ys
        
        #collision effects
        if wall[0] != None:
            if self.current_x <= wall[0]:
                self.current_x = wall[0]+1
        
        if wall[1] != None:
            
            #print("I am at x position ", str(self.current_x+self.character_width))
            
            if self.current_x+self.character_width >= wall[1]:
                self.current_x = wall[1]-self.character_width-1
                
            if self.current_x+self.character_width+1 >= wall[1]:
                    self.current_x -= self.wall_speed
        
        if self.current_y < self.floor:
            self.current_y = self.floor
        

In [49]:
###PLATFORMS###
class platform:
    def __init__(self, x1, x2, height):
        self.leftX = x1
        self.rightX = x2
        self.height = height

platform_generation_list = [(120,200,50)]

def generate_list_of_platform_objects(platform_generation_list):
    list_of_platforms = []
    for i in platform_generation_list:
        platform_temp = platform(i[0], i[1], i[2])
        list_of_platforms.append(platform_temp)
    return list_of_platforms

def generate_list_of_platforms_on_screen(list_of_platforms, virtual_window):
    list_of_platforms.sort(key = lambda plat: plat.leftX) ###SORT BY LEFTX
    list_of_platforms_on_screen = []
    for i in list_of_platforms:
        if i.rightX < 0:
            continue
        list_of_platforms_on_screen.append(i)
        if i.leftX > virtual_window[0]:
            break
    return list_of_platforms_on_screen

def blit_platform(platform, window_size, virtual_window):
    rectangle = (platform.leftX, platform.height - 5, platform.rightX-platform.leftX, 5)
    pixel_rectangle = xy_position_to_pixels(rectangle, window_size, virtual_window)
    screen.blit(texture_scaled,pixel_rectangle,pixel_rectangle)
    
    contrast_rectangle = (platform.leftX, platform.height - 3, platform.rightX-platform.leftX, 2)
    pixel_contrast_rectangle = xy_position_to_pixels(contrast_rectangle, window_size, virtual_window)
    AAfilledRoundedRect(screen,pixel_contrast_rectangle,(212, 78, 77),radius=0.4)
    
    icing_rectangle = (platform.leftX, platform.height - 2, platform.rightX-platform.leftX, 2)
    pixel_icing_rectangle = xy_position_to_pixels(icing_rectangle, window_size, virtual_window)
    AAfilledRoundedRect(screen,pixel_icing_rectangle,(255, 205, 205),radius=0.4)
    
    


###BLOCKS###
class block:
    def __init__(self, x, y, width, height):
        self.leftX = x
        self.rightX = x+width
        self.height = y + height
        self.bot = y
        

def generate_list_of_block_objects(block_generation_list):
    list_of_blocks = []
    for i in block_generation_list:
        block_temp = block(i[0],i[1], i[2], i[3])
        list_of_blocks.append(block_temp)
    return list_of_blocks

def generate_list_of_blocks_on_screen(list_of_blocks, virtual_window):
    list_of_blocks.sort(key = lambda bl: bl.leftX) ###SORT BY LEFTX
    list_of_blocks_on_screen = []
    for i in list_of_blocks:
        if i.rightX < 0:
            continue
        list_of_blocks_on_screen.append(i)
        if i.leftX > virtual_window[0]:
            break
    return list_of_blocks_on_screen

def blit_block(block, window_size, virtual_window):
    rectangle = (block.leftX, block.bot, block.rightX-block.leftX, block.height-block.bot)
    pixel_rectangle = xy_position_to_pixels(rectangle, window_size, virtual_window)
    screen.blit(texture_scaled,pixel_rectangle,pixel_rectangle)
    
    shadow_rectangle = (block.leftX, block.bot+block.height-4, block.rightX-block.leftX, 2)
    pixel_shadow_rectangle = xy_position_to_pixels(shadow_rectangle, window_size, virtual_window)
    AAfilledRoundedRect(screen,pixel_shadow_rectangle,(226, 168, 85),radius=0.4)
    
    contrast_rectangle = (block.leftX, block.bot+block.height-3, block.rightX-block.leftX, 2)
    pixel_contrast_rectangle = xy_position_to_pixels(contrast_rectangle, window_size, virtual_window)
    AAfilledRoundedRect(screen,pixel_contrast_rectangle,(212, 78, 77),radius=0.4)
    
    icing_rectangle = (block.leftX-2, block.bot+block.height-2, block.rightX-block.leftX+2, 2)
    pixel_icing_rectangle = xy_position_to_pixels(icing_rectangle, window_size, virtual_window)
    AAfilledRoundedRect(screen,pixel_icing_rectangle,(255, 205, 205),radius=0.4)
    
def scroll_everything_left(list_of_platforms, list_of_blocks, list_of_badbreads, world_speed):
    for platform in list_of_platforms:
        platform.rightX -= world_speed
        platform.leftX -= world_speed
    for block in list_of_blocks:
        block.rightX -= world_speed
        block.leftX -= world_speed
    for badbread in list_of_badbreads:
        badbread.startx -= world_speed
        badbread.endx -= world_speed
        badbread.current_bread_x -= world_speed

In [55]:
class badbread:
    def __init__(self, startx, starty, endx, speed = .5):
        self.startx = startx
        self.starty = starty
        self.endx = endx
        self.speed = speed
        self.current_bread_x = startx
    
    def move_badbread(self):
        if self.current_bread_x <= self.startx:
            self.speed = abs(self.speed)
        if self.current_bread_x >= self.endx:
            self.speed = -1*abs(self.speed)
        self.current_bread_x += self.speed

def generate_list_of_badbreads(badbread_generation_list):
    list_of_badbreads = []
    for i in badbread_generation_list:
        badbread_temp = badbread(i[0], i[1], i[2])
        list_of_badbreads.append(badbread_temp)
    return list_of_badbreads

def generate_list_of_badbreads_on_screen(list_of_badbreads, virtual_window):
    list_of_badbreads.sort(key = lambda bl: bl.startx) 
    list_of_badbreads_on_screen = []
    for i in list_of_badbreads:
        if i.endx+10 < 0:
            continue
        list_of_badbreads_on_screen.append(i)
        if i.startx > virtual_window[0]:
            break
    return list_of_badbreads_on_screen

def blit_badbread(badbread, window_size, virtual_window):
    rectangle = (badbread.current_bread_x, badbread.starty, 40, 40)
    pixel_rectangle = xy_position_to_pixels(rectangle, window_size, virtual_window)
    pygame.draw.rect(screen,(255,0,0),pixel_rectangle)



In [56]:
def give_floor_value(current_position, blocks_on_screen, platforms_on_screen):
    list_of_floors_under = []
    for i in blocks_on_screen:
        if i.rightX < current_position.current_x:
            continue
        if i.leftX > current_position.current_x+current_position.character_width:
            break
        if i.leftX < current_position.current_x+current_position.character_width and current_position.current_x < i.rightX:
            list_of_floors_under.append(i)
            
    for i in platforms_on_screen:
        if i.rightX < current_position.current_x:
            continue
        if i.leftX > current_position.current_x+current_position.character_width:
            break
        if i.leftX <= current_position.current_x+current_position.character_width and current_position.current_x <= i.rightX:
            list_of_floors_under.append(i)
    list_of_floors_under.sort(key = lambda structures: structures.height, reverse = True)
    if list_of_floors_under != []:
        list_of_floors_under.sort(key = lambda fl: fl.height, reverse = True)
        #print("y= ",current_position.current_y)
        for i in list_of_floors_under:
            if i.height <= current_position.current_y:
                #print("floor =",i.height)
                return i.height
    return -100

def give_wall_value(current_position, blocks_on_screen):
    level_with_character = []
    for i in blocks_on_screen:
        if i.bot > current_position.current_y:
            continue
        if i.height < current_position.current_y+current_position.character_height:
            continue
        level_with_character.append(i)
    closest_left = None
    closest_right = None
    if level_with_character != []:
        level_with_character.sort(key=lambda blocks: blocks.rightX)
        for i in level_with_character:
            if i.rightX < current_position.current_x:
                closest_left = i.rightX 
        level_with_character.sort(key=lambda blocks: blocks.leftX, reverse = True)
        for i in level_with_character:
            if i.leftX > current_position.current_x+current_position.character_width:
                closest_right = i.leftX 
    return (closest_left, closest_right)



In [57]:
def AAfilledRoundedRect(surface,rect,color,radius=0.4):

    """
    AAfilledRoundedRect(surface,rect,color,radius=0.4)

    surface : destination
    rect    : rectangle
    color   : rgb or rgba
    radius  : 0 <= radius <= 1
    """

    rect         = pygame.Rect(rect)
    color        = pygame.Color(*color)
    alpha        = color.a
    color.a      = 0
    pos          = rect.topleft
    rect.topleft = 0,0
    rectangle    = pygame.Surface(rect.size,pygame.SRCALPHA)

    circle       = pygame.Surface([min(rect.size)*3]*2,pygame.SRCALPHA)
    pygame.draw.ellipse(circle,(0,0,0),circle.get_rect(),0)
    circle       = pygame.transform.smoothscale(circle,[int(min(rect.size)*radius)]*2)

    radius              = rectangle.blit(circle,(0,0))
    radius.bottomright  = rect.bottomright
    rectangle.blit(circle,radius)
    radius.topright     = rect.topright
    rectangle.blit(circle,radius)
    radius.bottomleft   = rect.bottomleft
    rectangle.blit(circle,radius)

    rectangle.fill((0,0,0),rect.inflate(-radius.w,0))
    rectangle.fill((0,0,0),rect.inflate(0,-radius.h))

    rectangle.fill(color,special_flags= pygame.BLEND_RGBA_MAX)
    rectangle.fill((255,255,255,alpha),special_flags= pygame.BLEND_RGBA_MIN)

    return surface.blit(rectangle,pos)

In [54]:
virtual_window = (300,200)

pygame.init()
window_size = (pygame.display.Info().current_w -50, pygame.display.Info().current_h - 50)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Egg World")
done = False
clock = pygame.time.Clock()
BLUE = (0,0,255)

block_generation_list = [(0,0,300,20), (250,0,50, 50)]
wall = None
list_of_blocks = generate_list_of_block_objects(block_generation_list)
platform_generation_list = [(120,200,70)]
list_of_platforms = generate_list_of_platform_objects(platform_generation_list)
badbread_generation_list = [(60, 20, 90)]
list_of_badbreads = generate_list_of_badbreads(badbread_generation_list)


world_speed = .4
wall = None


current_position = position(current_y= virtual_window[1], wall_speed = world_speed)  


gradient = pygame.image.load('Gradient.jpeg')
gradient_scaled = pygame.transform.scale(gradient, window_size)
texture = pygame.image.load('CakeTexture.jpeg')
texture_scaled = pygame.transform.scale(texture, window_size)




#timer_event = pygame.USEREVENT + 1
#pygame.time.set_timer(timer_event, 250) #makes it so that your not waiting for events to run
while not done:
    scroll_everything_left(list_of_platforms, list_of_blocks, list_of_badbreads, world_speed)
    clock.tick(50)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                done = True
            #print("KEY DOWN!")
            current_position.button = keys(event)
            #print(current_position.button)
            break
        if event.type == pygame.KEYUP:
            #print("KEY UP!")
            current_position.button = None
            break
            
    blocks_on_screen = generate_list_of_blocks_on_screen(list_of_blocks, virtual_window)
    platforms_on_screen = generate_list_of_platforms_on_screen(list_of_platforms, virtual_window)
    wall = give_wall_value(current_position, blocks_on_screen)
    floor = give_floor_value(current_position, blocks_on_screen, platforms_on_screen)
    badbreads_on_screen = generate_list_of_badbreads_on_screen(list_of_badbreads, virtual_window)
    
    #print("Outside loop sees ", current_position.button)
    
    current_position.give_next_position(floor = floor, wall = wall)

    
    
    screen.blit(gradient_scaled,(0,0))


    ##floor
    for i in blocks_on_screen:
        blit_block(i, window_size, virtual_window)
    for i in platforms_on_screen:
        blit_platform(i, window_size, virtual_window)
    for i in badbreads_on_screen:
        i.move_badbread()
        blit_badbread(i, window_size, virtual_window)

    ##Character
    Rectangle = xy_position_to_pixels(
        (current_position.current_x, current_position.current_y,current_position.character_width,current_position.character_height), window_size, virtual_window)
    pygame.draw.rect(screen, BLUE, Rectangle)
    
    
    pygame.display.flip()
        
        
pygame.quit()

